In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np
import os

TensorFlow 2.x selected.


In [0]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Load the text files

In [0]:
if not os.path.exists('tok.valid.abstract.txt'):
  !wget -O 'tok.valid.abstract.txt' 'https://www.dropbox.com/s/0c6e9yf8yhf9a75/tok.valid.abstract.txt?dl=1'
if not os.path.exists('tok.valid.body.txt'):
  !wget -O 'tok.valid.title.txt' 'https://www.dropbox.com/s/aiy87847kusb7ju/tok.valid.title.txt?dl=1'

--2019-12-02 17:15:40--  https://www.dropbox.com/s/aiy87847kusb7ju/tok.valid.title.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/aiy87847kusb7ju/tok.valid.title.txt [following]
--2019-12-02 17:15:41--  https://www.dropbox.com/s/dl/aiy87847kusb7ju/tok.valid.title.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc327081136b3432f54e38b64e81.dl.dropboxusercontent.com/cd/0/get/AtcZwqoDyFwWPxOJwCH2kjrIyjlj7oShrWQ0nu77gmN21EPkmQVakqJTIS5MQ31R2Z4Dwk9u-xzaN2v9A1OyywrKO81SqyvqFgQ7U0swaYopFq3qyxADN8XZXr1LWtv3u4U/file?dl=1# [following]
--2019-12-02 17:15:41--  https://uc327081136b3432f54e38b64e81.dl.dropboxusercontent.com/cd/0/get/AtcZwqoDyFwWPxOJwCH2kjrIyjlj7oShrWQ0nu77gmN21EPkmQVakqJTIS5MQ31R2Z4Dwk9u-xzaN2v9A1OyywrK

In [0]:
with open('tok.valid.abstract.txt','rb') as f:
    body_data = f.read().decode("utf-8").split('\n')
    
with open('tok.valid.title.txt','rb') as f:
    target_data = f.read().decode("utf-8").split('\n')

Create vocabulary using keras tokenizer 

In [0]:
vocab_size = 20000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token='<UNK>')
tokenizer.fit_on_texts(body_data)
tokenizer.index_word[vocab_size] = '<s>'
tokenizer.index_word[vocab_size+1] = '<\s>'

In [0]:
class Vocab(object):
  def __init__(self, max_size):
    self._word_to_id = {}
    self._id_to_word = {}
    self._count = 0 # keeps track of total number of words in the Vocab
    self._word_to_id['<PAD>'] = self._count
    self._id_to_word[self._count] = "<PAD>"
    self._count += 1
    for _, word in tokenizer.index_word.items():
      if word in self._word_to_id:
        raise Exception('Duplicated word in vocabulary file: %s' % w)
      self._word_to_id[word] = self._count
      self._id_to_word[self._count] = word
      self._count += 1
      if max_size != 0 and self._count >= max_size:
        break
    print("Finished constructing vocabulary of %i total words. Last word added: %s" % (self._count, self._id_to_word[self._count-1]))
  def word2id(self, word):
    """Returns the id (integer) of a word (string). Returns [UNK] id if word is OOV."""
    if word not in self._word_to_id:
      return self._word_to_id['UNK']
    return self._word_to_id[word]

  def id2word(self, word_id):
    """Returns the word (string) corresponding to an id (integer)."""
    if word_id not in self._id_to_word:
      raise ValueError('Id not found in vocab: %d' % word_id)
    return self._id_to_word[word_id]

  def size(self):
    """Returns the total size of the vocabulary"""
    return self._count

In [0]:
vocab = Vocab(vocab_size+2)

Finished constructing vocabulary of 20002 total words. Last word added: <\s>


Convert text to numeric indicies

In [0]:
body_seqs=tokenizer.texts_to_sequences(body_data)
target_seqs=tokenizer.texts_to_sequences(target_data)

Add start and end tokens to all sequences. Start token index = vocab.size and end token index = vocab.size+1

In [0]:
body_seqs = [[vocab_size]+seq+[vocab_size+1] for seq in body_seqs]
target_seqs = [[vocab_size]+seq+[vocab_size+1] for seq in target_seqs]

Pad all sequences with zeros up to the maximum sequence length

In [0]:
max_len_body = max([len(seq) for seq in body_seqs])
max_len_target = max([len(seq) for seq in target_seqs])
body_seqs=tf.keras.preprocessing.sequence.pad_sequences(body_seqs, maxlen=max_len_body, padding="post")
target_seqs=tf.keras.preprocessing.sequence.pad_sequences(target_seqs, maxlen=max_len_target, padding="post")

In [0]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [0]:
import gzip
#if not os.path.exists('GoogleNews-vectors-negative300.bin.gz'):
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-12-02 17:15:51--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.88.221
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.88.221|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
import gensim

word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)  
embedding_matrix = np.zeros((vocab_size+2, 300))
for word, i in tokenizer.word_index.items():
    if i > vocab_size+2:
      break
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
file.download(embedding_matrix)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


NameError: ignored

Create a dataset from which batches of a certain size can be extracted

In [0]:
buffer_size = len(body_seqs)
batch_size = 16
train_dataset = tf.data.Dataset.from_tensor_slices((body_seqs,target_seqs))
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_units):
        super().__init__()
        
        self.hidden_units = hidden_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix])
        self.bi_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
            hidden_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform',
        ))
        
    def call(self, encoder_input,encoder_states):
        # inputs: encoder_input = (batch_size, seq_length)
        #         encoder_states = list[(batch_size, hidden_units),(batch_size, hidden_units)]
        
        # embedding look-up layer
        encoder_emb = self.embedding(encoder_input) # (batch_size,seq_length,embedding_dim)
        
        # encoder_output = (batch_size,seq_length,hidden_units)
        # encoder_states = (batch_size,hidden_units)
        encoder_output, state_fwd, state_back = self.bi_gru(encoder_emb,initial_state=encoder_states)
        encoder_states = [state_fwd,state_back]

        return encoder_output, encoder_states

In [0]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, hidden_units,is_coverage=False):
        super().__init__()
        
        self.Wh = tf.keras.layers.Dense(hidden_units) # weight matrix for encoder hidden state
        self.Ws = tf.keras.layers.Dense(hidden_units) # weight matrix for decoder state
        self.wc = tf.keras.layers.Dense(1) # weight vector for coverage
        self.V = tf.keras.layers.Dense(1)
        self.coverage = is_coverage
        if self.coverage is False:
            self.wc.trainable = False
        
    def call(self, decoder_state, encoder_output,coverage_vector):
        # inputs: decoder_state = (batch_size, hidden_units)
        #         encoder_output = (batch_size, seq_length, hidden_units)
        #         coverage_vector = (batch_size, seq_length)

        # expand dimension of decoder state and coverage vector to allow addition
        decoder_state = tf.expand_dims(decoder_state, 1) # (batch_size, 1, hidden_units)
        coverage_vector = tf.expand_dims(coverage_vector, 1) # (batch_size, 1, seq_length)

        # calculate attention scores
        # score = (batch_size, length, 1)
        score = self.V(tf.nn.tanh(
                        self.Wh(encoder_output) +  # (batch_size, length, hidden_units) -> (batch_size, length, attention_units)
                        self.Ws(decoder_state) +  # (batch_size, 1, hidden_units) -> (batch_size, 1, attention_units)
                        self.wc(coverage_vector) # (batch_size, 1, seq_length) -> (batch_size, 1, 1)
                        )) 
        
        attention_weights = tf.nn.softmax(score, axis=1) # (batch_size, seq_length, 1)
        # only update coverage vector if coverage is enabled
        coverage_vector = tf.squeeze(coverage_vector,1) # (batch_size, seq_length)
        if self.coverage is True:
          coverage_vector+=tf.squeeze(attention_weights) 

        context_vector = attention_weights * encoder_output # (batch_size, seq_length, hidden_units)
        context_vector = tf.reduce_sum(context_vector, axis=1) # (batch_size, hidden_units)

        return context_vector, attention_weights, coverage_vector

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_units):
        super().__init__()
        
        self.hidden_units = hidden_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix])
        self.gru = tf.keras.layers.GRU(
            hidden_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform',
        )
        self.W1 = tf.keras.layers.Dense(hidden_units)
        self.W2 = tf.keras.layers.Dense(vocab_size)
        # Pointer Generator
        # wh = tf.keras.layers.Dense(1)
        # ws = tf.keras.layers.Dense(1)
        # wx = tf.keras.layers.Dense(1)

        
    def call(self, decoder_input, decoder_state, encoder_output,context_vector):
        # inputs: decoder_input = (batch_size, 1)
        #         decoder_state = (batch_size, hidden_units)
        #         encoder_output = (batch_size,seq_length, hidden_units)
        #         coverage_vector = (batch_size,seq_length)

        # embedding look-up layer
        decoder_emb = self.embedding(decoder_input) # (batch_size, seq_length, hidden_units)

        # decoder_output = (batch_size,seq_length,hidden_units)
        # decoder_state = (batch_size,hidden_units)
        decoder_output , decoder_state = self.gru(decoder_emb,initial_state=decoder_state)

        # concatenate context vector and decoder state 
        concat_vector = tf.concat([context_vector,decoder_state], axis=-1)
        # reshape to 1d array
        concat_vector = tf.reshape(concat_vector, (-1, concat_vector.shape[1]))
        # create vocabulary distribution
        p_vocab = tf.nn.log_softmax(self.W2(self.W1(concat_vector)))

        # calculate p_gen
        #p_gen = tf.nn.sigmoid(self.wh(context_vector)+self.ws(decoder_state)+self.wx(decoder_input))
          
        return p_vocab, decoder_state

Initialize encoder and decoder

In [0]:
embedding_dim = 300
hidden_units = 128
encoder = Encoder(tokenizer.num_words+2, embedding_dim, hidden_units) # +2 on due to start and end tokens
attention = BahdanauAttention(hidden_units,is_coverage=True)
decoder = Decoder(tokenizer.num_words+2, embedding_dim, hidden_units)

Run one batch through the model in order to initialize model parameters

In [0]:
encoder_input, decoder_target = next(iter(train_dataset))
encoder_init_states = [tf.zeros((batch_size, encoder.hidden_units)) for i in range(2)]
encoder_output, encoder_states = encoder(encoder_input,encoder_init_states)
decoder_state = encoder_states[0] 
coverage_vector = tf.zeros((16,encoder_input.shape[1]))
decoder_input_t = decoder_target[:,0]
context_vector, attention_weights, coverage_vector = attention(decoder_state, encoder_output,coverage_vector)
p_vocab,decoder_state = decoder(tf.expand_dims(decoder_input_t,1),decoder_state,encoder_output,context_vector)

Load pretrained weights if needed

In [0]:
# model_save_path = ""
# encoder_save_name = "encoder_weights30.h5"
# decoder_save_name = "decoder_weights30.h5"
# attention_save_name = "attention_30epochs.h5"
# encoder.load_weights(os.path.join(model_save_path,encoder_save_name))
# decoder.load_weight(os.path.join(model_save_path,decoder_save_name))
# attention.load_weights(os.path.join(model_save_path,attention_save_name))

Define optimizer and loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()

def nll_loss(p_vocab,target):
    # apply a mask such that pad zeros do not affect the loss
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss = -p_vocab
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask  
    return loss

def coverage_loss(attention_weights,coverage_vector,target):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    coverage_vector = tf.expand_dims(coverage_vector,axis=2)
    ct_min = tf.reduce_min(tf.concat([attention_weights,coverage_vector],axis=2),axis=2)
    cov_loss = tf.reduce_sum(ct_min,axis=1)
    mask = tf.cast(mask, dtype=cov_loss.dtype)
    cov_loss *= mask
    return cov_loss

Define a function for performing one training step (one batch)

In [0]:
@tf.function
def train_step(encoder_input, decoder_target):
    """Function which performs one training step (batch)"""
    loss = tf.zeros(batch_size)
    lambda_cov = 1
    with tf.GradientTape() as tape:
        # run body_sequence input through encoder
        encoder_init_states = [tf.zeros((batch_size, encoder.hidden_units)) for i in range(2)]
        encoder_output, encoder_states = encoder(encoder_input,encoder_init_states)
        # initialize decoder with encoder forward state
        decoder_state = encoder_states[0] # !!!interpolate between forward and backward instead!!!
        coverage_vector = tf.zeros((16,encoder_input.shape[1]))
        # loop over each word in target sequence
        for t in range(decoder_target.shape[1]-1):
            # run decoder input through decoder and generate vocabulary distribution
            decoder_input_t = decoder_target[:,t]
            decoder_target_t = decoder_target[:,t+1]
            # get attention scores
            context_vector, attention_weights, coverage_vector = attention(decoder_state, encoder_output,coverage_vector)
            # get vocabulary distribution for each batch at time t
            p_vocab,decoder_state = decoder(tf.expand_dims(decoder_input_t,1),decoder_state,encoder_output,context_vector)
            # for each batch get the probability of the target word at time t+1
            p_vocab_list = []
            for i in range(len(decoder_target_t)):
                p_vocab_list.append(p_vocab[i,decoder_target_t[i]])
            p_vocab_target = tf.stack(p_vocab_list)
            # calculate the loss at each time step t and add to current loss
            loss += nll_loss(p_vocab_target,decoder_target_t) + lambda_cov*coverage_loss(attention_weights,coverage_vector,decoder_target_t)

        # get the non-padded length of each sequence in the batch
        seq_len_mask = tf.cast(tf.math.logical_not(tf.math.equal(decoder_target, 0)),tf.float32)
        batch_seq_len = tf.reduce_sum(seq_len_mask,axis=1)

        # get batch loss by dividing the loss of each batch by the target sequence length and mean
        batch_loss = tf.reduce_mean(loss/batch_seq_len)

    # update trainable variables
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(batch_loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

Training loop

In [0]:
from tqdm import tqdm_notebook as tqdm

epochs = 10

epoch_loss = tf.keras.metrics.Mean()
with tqdm(total=epochs) as epoch_progress:
    for epoch in range(epochs):
        epoch_loss.reset_states()

        with tqdm(total=len(body_seqs) // batch_size) as batch_progress:
            for batch, (encoder_input, decoder_target) in enumerate(train_dataset):
                batch_loss = train_step(encoder_input, decoder_target)
                epoch_loss(batch_loss)
                
                if (batch % 10) == 0:
                    batch_progress.set_description(f'Epoch {epoch + 1}')
                    batch_progress.set_postfix(Batch=batch, Loss=batch_loss.numpy())
                batch_progress.update()
        
        epoch_progress.set_description(f'Epoch {epoch + 1}')
        epoch_progress.set_postfix(Loss=epoch_loss.result().numpy())
        epoch_progress.update()

Evaluate model

In [0]:
def greedy_search(encoder_input,max_sum_len = 20):
    """Function which returns a summary by always picking the highest probability option conditioned on the previous word"""
    # run body_sequence through encoder
    encoder_init_states = [tf.zeros((1, encoder.hidden_units)) for i in range(2)]
    encoder_output, encoder_states = encoder(encoder_input,encoder_init_states)
    # initialize decoder with encoder forward state
    decoder_state = encoder_states[0]

    decoder_input_t = tf.ones(1)*tokenizer.num_words # initialize with start token
    summary = [tokenizer.num_words]
    coverage_vector = tf.zeros((1,encoder_input.shape[1]))
    while decoder_input_t[0].numpy()!=(tokenizer.num_words+1) and len(summary)<max_sum_len: # as long as decoder input is different from end token continue
        context_vector, attention_weights, coverage_vector = attention(decoder_state, encoder_output,coverage_vector)
        p_vocab, decoder_state = decoder(tf.expand_dims(decoder_input_t,1),decoder_state,encoder_output,context_vector)
        decoder_input_t = tf.argmax(p_vocab,axis=1) 
        decoder_word_idx = int(decoder_input_t[0].numpy())
        summary.append(decoder_word_idx)
    return summary

In [0]:
def best_k_candidates(prev,k,encoder_output):
    # use decoder to generate probability over vocabulary for the next token
    decoder_input_t = tf.expand_dims(tf.expand_dims(prev[0][-1],0),1)
    context_vector, attention_weights, coverage_vector = attention(prev[2][0], encoder_output,prev[2][1])
    p_vocab, decoder_state = decoder(decoder_input_t, prev[2][0],encoder_output,context_vector)    
    # sort the hypothesis by probability
    idx = tf.argsort(p_vocab, direction='DESCENDING')  # sorted indices
    ranks = tf.argsort(idx, direction='ASCENDING')  # ranks
    filter_k = ranks < k # return True in the position with rank 1, 2, ..., k
    # just for convenience, change to numpy
    size = vocab_size+2 
    p_vocab = p_vocab.numpy().reshape(size,)
    filter_k = filter_k.numpy().reshape(size,)
    # get the best hypothesis
    best_k_candidates = [ [prev[0]+[x],p_vocab[x]+prev[1],[decoder_state,coverage_vector]] for x in range(size) if filter_k[x] ]
    return best_k_candidates

In [0]:
def best_k_of_k2(best_k,k,completed,encoder_output):
    results = []
    # for each previous hypothesis, find k best hypothesis induced from it. THis will result in k^2 hypothesis
    for hypo in best_k:
      results = results + best_k_candidates(hypo, k, encoder_output)
    # sort the list and extract k best hypothesis
    results = sorted(results,key = lambda x: x[1],reverse= True)[0:k]
    # if there is a completed hypothesis (end token generated), transfer it to completed set and decrease the beam size
    for result in results:  
      if result[0][-1] == vocab_size+1:
        k-=1
        results.remove(result)
        completed += [result]
    return results,k,completed

In [0]:
def beam_search(encoder_input, max_sum_len = 20, beam_width = 5):
      # run body_sequence input through encoder
      encoder_init_states = [tf.zeros((1, encoder.hidden_units)) for i in range(2)]
      encoder_output, encoder_states = encoder(encoder_input,encoder_init_states)

      # initialize decoder with encoder forward states
      decoder_states = encoder_states[0]
      coverage_vector = tf.zeros((1,encoder_input.shape[1]))
      # initialize the hypothesis: [sequence <s>, log probability 0, decoder states]
      prev = [[vocab_size],0,[decoder_states,coverage_vector]]
      # get the beam size and create a list to store completed hypothesis 
      k = beam_width
      completed = []
      # get k best first token
      best_k = best_k_candidates(prev,k,encoder_output)

      # use beam search for max_sum_len (maximum length) steps
      for i in range(max_sum_len):
        # get k best hypothesis when adding a new token
        best_k,k,completed = best_k_of_k2(best_k,k,completed,encoder_output)
        # stop when there are enough completed hypothesis
        if len(completed) == k:
          break 
      
      # when there are no completed hypothesis, take 5 last hypothesis as the final candidates
      if len(completed) == 0:
        completed = best_k
      # normalized the hypothesis probability by the length of hypothesis
      for hypo in completed:
        hypo[1]/=len(hypo[0])
        hypo = [hypo[1],hypo[2]]   
      # sort the hypothesis by normalized probability and choose the best one       
      best_k = sorted(completed,key=lambda x: x[1],reverse=True)[0][0]
      return best_k

In [0]:
encoder_save_name = "encoder_30epochs.h5"
decoder_save_name = "decoder_30epochs.h5"
attention_save_name = "attention_30epochs.h5"
encoder.save_weights(encoder_save_name)
decoder.save_weights(decoder_save_name)
attention.save_weights(attention_save_name)

In [0]:
# files.download(attention_save_name)

In [0]:
encoder_input, decoder_target = next(iter(train_dataset))

In [0]:
encoder_input_sum = tf.expand_dims(encoder_input[0,:],0)
summary_greedy = greedy_search(encoder_input_sum)
summary_beam = beam_search(encoder_input_sum)

In [0]:
target_summary = [d for d in decoder_target.numpy()[0] if d!=0]

In [0]:
print("Generated by greedy search:"+" ".join([tokenizer.index_word[idx] for idx in summary_greedy]))
print("Generated by beam search:"+" ".join([tokenizer.index_word[idx] for idx in summary_beam]))
print("Target:"+" ".join([tokenizer.index_word[idx] for idx in target_summary]))